In [4]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")
os.environ["TAVILY_API_KEY"]=os.getenv("TAVILY_API_KEY")


In [5]:
from langchain_groq import ChatGroq

llm=ChatGroq(model="llama-3.1-8b-instant")
response = llm.invoke("Hello! How are you today?")
print(response.content)

I'm doing well, thank you for asking. I'm a large language model, so I don't have emotions or feelings like humans do, but I'm functioning properly and ready to assist you with any questions or topics you'd like to discuss. How can I help you today?


In [ ]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_ollama import OllamaEmbeddings

embd=(OllamaEmbeddings(model="mxbai-embed-large:335m"))

urls=[
    "https://lilianweng.github.io/posts/2023-06-23-agent/",
    "https://lilianweng.github.io/posts/2023-03-15-prompt-engineering/",
    "https://lilianweng.github.io/posts/2023-10-25-adv-attack-llm/"

]

docs= [WebBaseLoader(url).load() for url in urls]

# docs_list = []
# for url in urls:
#     loaded_docs = WebBaseLoader(url).load()
#     docs_list.extend(loaded_docs)

docs_list=[items for sublist in docs for items in sublist]

# docs_list = []
# for sublist in docs:
#     for item in sublist:
#         docs_list.append(item)

text_splitter=RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=1000,chunk_overlap=0
)
doc_splits=text_splitter.split_documents(docs_list)

#Add Vector Store
vector_storage=FAISS.from_documents(
    documents=doc_splits,
    embedding=embd
)



In [10]:
retriver=vector_storage.as_retriever()

print("Vector store created successfully!")
print(f"Number of original documents: {len(docs_list)}")
print(f"Number of document chunks (vectors): {len(doc_splits)}")

Vector store created successfully!
Number of original documents: 3
Number of document chunks (vectors): 44


In [ ]:
from langchain_core.prompts import ChatMessagePromptTemplate
from pydantic import BaseModel , Field

def Grade_Doc(BaseModel):
    """Binary Score for realtive check on retrived documents"""

    binary_score=Field(
        description="Documnet sare relevent to the question 'yes' or 'no'"
    )

structured_llm_grader=llm.with_structured_output(Grade_Doc)
    